In [15]:
import numpy as np
import pandas as pd
import matplotlib as plt #ne pas semettre
import seaborn as sns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import re
import nltk
import spacy
import gensim
from nltk.corpus import stopwords

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

import pickle
import unidecode

In [55]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


# Extraction des Tags

In [3]:
#df1
post_df= pd.read_csv("Posts.csv").dropna(subset=['Title', 'Tags'])
df1 = post_df[(post_df["AnswerCount"].abs() > 10) & \
            (post_df["FavoriteCount"].abs() >  200) & \
            (post_df["ViewCount"].abs() > 5000)   & \
            (post_df["Score"].abs() > 15)]
#df2
PostsWithDeleted_df= pd.read_csv("PostsWithDeleted.csv").dropna(subset=['Title', 'Tags'])
df2 = PostsWithDeleted_df[(PostsWithDeleted_df["AnswerCount"].abs() > 10) & \
            (PostsWithDeleted_df["FavoriteCount"].abs() >  200) & \
            (PostsWithDeleted_df["ViewCount"].abs() > 5000)   & \
            (PostsWithDeleted_df["Score"].abs() > 15)]
#df3
df3= pd.read_csv("SuggestedEdits.csv").dropna(subset=['Title', 'Tags'])

#df4
df4= pd.read_csv("TagSynonyms.csv").dropna(subset=['SourceTagName', 'TargetTagName'])
df4 = df4.rename(columns={"TargetTagName": "New_Tags"})

#concat
df = pd.concat([df1[["Title", "Tags"]], df2[["Title", "Tags"]],df3[["Title", "Tags"]]])
df["Tags"] = df["Tags"].astype(str)
df = df.reset_index(drop=True)
print(df.shape)


#to select the first tag
def extract_tags(string_to_change):
    liste = []
    for element in string_to_change:
        #drop the tags < >
        test = re.sub('><|<|>', ' ', element)
 
        #string to list of strings
        list_split = test.split()

        #list with the first zzz
        res = list_split[0]
   
        #with / separator 
        #res_join= '/'.join(res)
    
        #store results
        liste.append(res)
        #print(liste)
        
    s = pd.Series(liste)
    s = s.astype(str)
        
    return s
df["New_Tags"] = extract_tags(df["Tags"])

#Merge with existing tags 
df_merged = df.merge(df4, how='inner', on='New_Tags').drop_duplicates(subset=['Title',"New_Tags"])
df_droped = df_merged.drop(df_merged.iloc[:, 3:13], axis=1)
df_final = df_droped.drop(labels=['Tags'], axis=1)

df_final["Title"] = df_final["Title"].astype(str)
df_final = df_final.dropna(axis=0)

df_final = df_final.reset_index(drop=True)
df_final.head()

(2015, 2)


,Title,New_Tags
0,How do I calculate someone's age based on a Da...,c#
1,Calculate relative time in C#,c#
2,What Are Some Good .NET Profilers?,c#
3,What is the difference between String and stri...,c#
4,What does the [Flags] Enum Attribute mean in C#?,c#


# Normalisation des Titles

In [4]:
#TOKENIZER
data = df_final.Title.values.tolist()

def sent_to_words(sentences):
    
    '''
    sentences : list of sentences
    '''
    
    sentences = [re.sub(r"\'|/|<|>", " ", sent) for sent in sentences]
    sentences = [re.sub(r"#", 'diese', sent) for sent in sentences]
    sentences = [re.sub(r"\+", 'plus', sent) for sent in sentences]
    
   
    for sentence in sentences:
                
        # deacc= True removes punctuations
        #lowercase
        #remove one caracter only
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

#data_words = list(sent_to_words(data))
#data_words #> List of list of words

In [5]:
#STOPWORDS
nltk.corpus.stopwords.words('english')

def stop_words(texts):
    '''
    texts : list of words - data_words
    corpus: list of sentences
    '''

    corpus = []
    for i in range(0,len(texts)):
        title = texts[i]
        #review = review.split() #list of words
        title = [word for word in title if not word in set(stopwords.words('english'))]
        title = ' '.join(title) #list of words to one sentence
        corpus.append(title)
        
    return corpus   

#stop_words(data_words)      #> list of sentences

In [6]:
#LEMMATIZATION
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
                   #data_words
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB']): 
    '''
    texts : list of words
    '''
    texts_out = []
    for sent in texts:
        #words to sentence
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

#lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB'])

In [7]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data = df_final.Title.values.tolist()

def clean_titles(text, nlp=nlp):
    global sent_to_words
    global lemmatization
    '''
    text : list of sentence(s)
    '''

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text)) #> list of words
    
    #Step1_0 : stopwords
    mytext_3 = stop_words(mytext_2) #> list of sentences
    
    # Step 2: Lemmatize      
    liste=[]

    for element in mytext_3:
        title = element.split()
        liste.append(title)
        
        mytext_3 = liste #> list of words
        
                            #mytext_2 = data_words
    mytext_4 = lemmatization(mytext_3, allowed_postags=['NOUN', 'ADJ', 'VERB']) #> list of sentences

    #mytext_5 = count_SVD_vectorizer.transform(mytext_4)
    
    
    return mytext_4

clean_titles(data, nlp=nlp)

['calculate age base datetime type birthday',
 'calculate relative time cdiese',
 'good net profiler',
 'difference string string cdiese',
 'flag enum attribute mean cdiese',
 'hide feature cdiese',
 'linq query datatable',
 'pro con keep sql store proc code',
 'read file',
 'correct way create single instance wpf application',
 'cast int enum',
 'difference generic cdiese java template cplusplus',
 'send email net gmail',
 'yield use cdiese',
 'good way give cdiese auto property initial value',
 'dynamic linq orderby ienumerable iqueryable',
 'use',
 'get path assembly code',
 'difference const cdiese',
 'deep cloning object',
 'create generic method constrain enum',
 'pass user credential sharepoint use sp object model web service uploading file',
 'show object gridview usful control',
 'pass dictionary string string object method',
 'cast unbox conversion',
 'console input integer',
 'use interface service contract concrete type wcf',
 'dial number nokia phone use cdiese',
 'create 

In [8]:
#list to series
series = pd.Series(clean_titles(data, nlp=nlp), name = "Title")
series = series.astype('str', copy=False)

#series to df["Title"]
df_cleaned = series.to_frame()
df_cleaned.head()

,Title
0,calculate age base datetime type birthday
1,calculate relative time cdiese
2,good net profiler
3,difference string string cdiese
4,flag enum attribute mean cdiese


In [9]:
#Add the New_Tads column
df_cleaned["New_Tags"] = df_final["New_Tags"] 
df_cleaned.head()

,Title,New_Tags
0,calculate age base datetime type birthday,c#
1,calculate relative time cdiese,c#
2,good net profiler,c#
3,difference string string cdiese,c#
4,flag enum attribute mean cdiese,c#


# Split Data

In [10]:
X = df_cleaned['Title'].to_numpy()
y = df_cleaned['New_Tags'].to_numpy()

#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

print(df_cleaned.shape)
df_cleaned.head()

(1709, 2)


,Title,New_Tags
0,calculate age base datetime type birthday,c#
1,calculate relative time cdiese,c#
2,good net profiler,c#
3,difference string string cdiese,c#
4,flag enum attribute mean cdiese,c#


In [11]:
X_train

array(['print alternate line use python',
       'overload action number parameter asp net mvc',
       'edittext value store database', ..., 'verify signature hash',
       'voice text application wifi', 'rail habtm validation'],
      dtype=object)

In [12]:
X_test

array(['batch append date time creation random end filename',
       'app purchasing remove iad banner ipad app',
       'pass specific attribute object store array tableview cell',
       'retrieve value array', 'compatible javascript css file',
       'get setting button display live wallpaper',
       'hide feature cplusplus', 'use enough secure query string',
       'possible filter user queue appear view assigning case',
       'mysql profile tool', 'javascript query string work',
       'grab recent search',
       'entity model issue able create model particular table',
       'alternative cursor sql loop', 'break format nsstre multiple line',
       'deploy web testing config build server',
       'ajax loading screen prototype unobtrusive',
       'run macro access web database application instance initialize',
       'email send server', 'pass dictionary string string object method',
       'dialog logout access token facebook io',
       'prevent disabled jmenuitem hiding me

In [13]:
y_train

array(['python', 'asp.net-mvc', 'android', ..., 'java', 'android',
       'ruby-on-rails'], dtype=object)

In [14]:
y_test

array(['windows', 'iphone', 'objective-c', 'c#', 'javascript', 'android',
       'c++', 'php', 'plugins', 'mysql', 'javascript', 'php', 'error',
       'sql', 'iphone', 'web-config', 'ruby-on-rails', 'database', 'php',
       'c#', 'iphone', 'java', 'javascript', 'ruby-on-rails',
       'calculated-columns', 'svn', 'homework', 'c#', 'database',
       'jquery', 'python', 'iphone', 'security', 'javascript',
       'javascript', 'javascript', 'assembly', 'jquery', 'php', 'php',
       'javascript', 'android', 'java', 'java', 'c++', 'python',
       'objective-c', 'ios', 'javascript', 'jquery', 'python', 'vb.net',
       'git', 'c#', 'c++', 'php', 'html', 'asp.net', 'php', 'c#', 'ios',
       'php', 'javascript', 'php', 'android', 'iphone', 'iphone', 'regex',
       'vb.net', 'iphone', 'c#', 'php', 'javascript', 'c#', 'c#', 'c#',
       'facebook', 'javascript', 'iphone', 'java', 'php', 'java',
       'delphi', 'c#', 'vb.net', 'wcf', 'javascript', 'c++', 'android',
       'iphone', 'iphon

# Version1 - Essai avec Pipeline 

In [16]:
#Vectorizer
count_SVD_vectorizer = make_pipeline(CountVectorizer(analyzer='word',
                                                     # minimum occurences of a word at leat 10 times
                                                     min_df =10,    
                                                     # remove stop words
                                                     stop_words='english',
                                                     # convert all words to lowercase
                                                     lowercase=False,     
                                                     # # num chars > 2 
                                                     token_pattern='[a-zA-Z0-9]{2,}',  # num chars > 3 
                                                     
                                                     #GridSearch: max number of uniq words    
                                                      max_features=50000), 
                                     #n_components must be < n_features = 112
                                     TruncatedSVD(n_components=100)) 

count_SVD_vectorizer.fit(X_train)
feat_train = count_SVD_vectorizer.transform(X_train)
feat_test = count_SVD_vectorizer.transform(X_test)

#Model
lr = LogisticRegression(multi_class='ovr', random_state = 42) 
params = {'C': np.logspace(-3, 3, 7) , 'penalty':['l1','l2'] }

#train the gs model
lr_gs = GridSearchCV(lr, params, cv=None)#None, to use the default 5-fold cross validation,
lr_gs.fit(feat_train, y_train)

#train the log model
best_lr_model = lr_gs.best_estimator_

/home/catherinele/.local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/home/catherinele/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/catherinele/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    r

/home/catherinele/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/catherinele/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/catherinele/.local/lib/

/home/catherinele/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/catherinele/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/catherinele/.local/lib/

/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

...

/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/catherinele/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

In [17]:
from sklearn.pipeline import Pipeline


#transformer = le.inverse_transform()


pipeline = Pipeline(
    [('count', CountVectorizer(analyzer='word',
                               # minimum occurences of a word at leat 10 times
                               min_df =10,    
                               # remove stop words
                               stop_words='english',
                               # convert all words to lowercase
                               lowercase=True,     
                               # # num chars > 2 
                               token_pattern='[a-zA-Z0-9]{2,}',  # num chars > 3 
                               
                               #GridSearch: max number of uniq words    
                                max_features=50000)),
    ("SVD", TruncatedSVD(n_components=100) ),
    ("clf", best_lr_model)
    ])

In [25]:
#train and test the best_lr_model 
pipeline.fit(X_train,y_train) 
y_pred = pipeline.predict(X_test)
y_pred

array(['c#', 'java', 'java', 'objective-c', 'javascript', 'iphone', 'c++',
       'c#', 'iphone', 'mysql', 'javascript', 'iphone', 'c#', 'sql',
       'iphone', 'java', 'java', 'database', 'php', 'java', 'c#', 'java',
       'iphone', 'java', 'sql', 'iphone', 'java', 'asp.net',
       'objective-c', 'java', 'objective-c', 'iphone', 'c#', 'javascript',
       'javascript', 'javascript', 'java', 'iphone', 'php', 'objective-c',
       'jquery', 'java', 'php', 'java', 'c++', 'java', 'c#', 'iphone',
       'javascript', 'jquery', 'iphone', 'c#', 'java', 'c#', 'java',
       'php', 'c#', 'asp.net', 'database', 'java', 'iphone', 'html',
       'java', 'java', 'android', 'iphone', 'iphone', 'java', 'c#',
       'iphone', 'php', 'java', 'c#', 'c#', 'c#', 'c#', 'sql', 'c#',
       'java', 'java', 'c#', 'java', 'c#', 'c#', 'c#', 'c#', 'javascript',
       'objective-c', 'c#', 'iphone', 'iphone', 'html', 'java', 'java',
       'java', 'c#', 'java', 'java', 'c#', 'iphone', 'mysql', 'java',
       '

In [27]:
# test on different type of inputs
print(pipeline.predict(["read write datum database"])[0])
print(pipeline.predict(np.array(["read write datum database"]))[0] )
print(pipeline.predict(np.array(["read write datum database"]).reshape(-1))[0])

java
java
java


In [29]:
#save and load the pipeline
pickle.dump(pipeline, open('pipeline.pkl', 'wb'))
pickled_pipeline = pickle.load(open('pipeline.pkl', 'rb'))

y_pred = pickled_pipeline.predict(X_test)
y_pred

array(['c#', 'java', 'java', 'objective-c', 'javascript', 'iphone', 'c++',
       'c#', 'iphone', 'mysql', 'javascript', 'iphone', 'c#', 'sql',
       'iphone', 'java', 'java', 'database', 'php', 'java', 'c#', 'java',
       'iphone', 'java', 'sql', 'iphone', 'java', 'asp.net',
       'objective-c', 'java', 'objective-c', 'iphone', 'c#', 'javascript',
       'javascript', 'javascript', 'java', 'iphone', 'php', 'objective-c',
       'jquery', 'java', 'php', 'java', 'c++', 'java', 'c#', 'iphone',
       'javascript', 'jquery', 'iphone', 'c#', 'java', 'c#', 'java',
       'php', 'c#', 'asp.net', 'database', 'java', 'iphone', 'html',
       'java', 'java', 'android', 'iphone', 'iphone', 'java', 'c#',
       'iphone', 'php', 'java', 'c#', 'c#', 'c#', 'c#', 'sql', 'c#',
       'java', 'java', 'c#', 'java', 'c#', 'c#', 'c#', 'c#', 'javascript',
       'objective-c', 'c#', 'iphone', 'iphone', 'html', 'java', 'java',
       'java', 'c#', 'java', 'java', 'c#', 'iphone', 'mysql', 'java',
       '

In [30]:
# test the loaded model on different type of inputs
print(pickled_pipeline.predict(["use enough secure query string"])[0])
print(pickled_pipeline.predict(np.array(["use enough secure query string"]))[0])
print(pickled_pipeline.predict(np.array(["use enough secure query string"]).reshape(-1))[0])

c#
c#
c#


**Sources:**

Installation Flask on Linux: 

https://linuxize.com/post/how-to-install-flask-on-ubuntu-20-04/